In [1]:
import pandas as pd
import datetime
import numpy as np
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import pickle

In [2]:
directory = 'MLP_Classifiers_480k_training_60_iter_NN_size_200_200_200_50_30_30'

In [3]:
train1 = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_train_1.csv')
train2 = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_train_2.csv')
train = pd.concat([train1, train2])
train = train[train.notnull().apply(all, axis=1)]
dev = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_dev.csv')
dev = dev[dev.notnull().apply(all, axis=1)]
test = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_test.csv')
test = test[test.notnull().apply(all, axis=1)]

dev_prepared = pd.read_csv('../02_Extract_Subphrases/prepared_data/subphrase_vectors_dev.csv', sep=';')
dev_prepared = dev_prepared.drop(columns='Unnamed: 0')
dev = dev.set_index('pairID')
rel_pairIDs = dev_prepared.iloc[:,0]
y_hat = dev.loc[rel_pairIDs].gold_label
dev_prepared = dev_prepared.iloc[:,1:].to_numpy()

In [4]:
dev_subphrases = pd.read_csv('../02_Extract_Subphrases/prepared_data/subphrases_dev.csv', sep=',')
dev_subphrases = dev_subphrases.set_index('pairID')
dev_subphrases = dev_subphrases.loc[rel_pairIDs]

In [5]:
dev_subphrases.iloc[0]['string_subj_s1']

'Two women'

In [6]:
def evaluate_performance(preds, actual):
    labels = ['contradiction', 'entailment', 'neutral']
    print('Confusion Matrix')
    print(confusion_matrix(actual, preds, labels=labels))
    print(f'F1-Score: {f1_score(actual, preds, labels=labels, average="macro")}')
    print(f'Accuracy: {accuracy_score(actual, preds)}')

In [7]:
def predict_y_from_z(z):
    if len(z.shape) > 1:
        z = pd.DataFrame(z)
        res = z.apply(predict_y_from_z, axis=1)
        return res.to_numpy()
    else:
        if all([z[i] == 'nan' or pd.isnull(z[i]) or z[i] == 'entailment' for i in range(25)]):
            return 'entailment'
        elif any(z == 'contradiction'):
            return 'contradiction'
        elif any(z == 'neutral') and all(z != 'contradiction'):
            return 'neutral'
        else:
            raise ValueError(f"z can only have values 'entailment', 'contradiction', or 'neutral' but is {z}")

In [8]:
clf = list()
for i in range(25):
    with open("../03_Bayesian_Network/" + directory + "/MLP_Classifier" + str(i) + ".pkl", "rb") as f:
        clf += [pickle.load(f), ]

In [9]:
# Prepare colum indices
indices = [[0,1500], [0,1800], [0,2100], [0,2400], [0,2700],
           [300,1500], [300,1800], [300,2100], [300,2400], [300,2700],
           [600,1500], [600,1800], [600,2100], [600,2400], [600,2700],
           [900,1500], [900,1800], [900,2100], [900,2400], [900,2700],
           [1200,1500], [1200,1800], [1200,2100], [1200,2400], [1200,2700]]

# Initialise colulmn indices and "nan" values if information (e.g. location of sentence) is not detected
print(datetime.datetime.now())
print("Initialise column indices and 'nan' values")
not_nan = [None, ] * 25
cols = [None, ] * 25
for i in range(25):
    cols[i] = list(range(indices[i][0], indices[i][0]+300)) + list(range(indices[i][1],indices[i][1]+300))
    not_nan[i] = pd.Series([not x for x in pd.DataFrame(np.isnan(dev_prepared[:,cols[i]])).apply(any, axis=1)])
not_nan = np.array(not_nan).T

2023-03-09 09:49:24.762499
Initialise column indices and 'nan' values


In [10]:
z = np.empty((y_hat.shape[0], 25), dtype=np.dtype('U100'))

z[:,:] = np.nan

for i in range(25):
    z[not_nan[:,i], i] = clf[i].predict(dev_prepared[not_nan[:,i],:][:, cols[i]])

y_hat_pred = predict_y_from_z(z)

In [96]:
def generate_explanation(Z, subphrases):
    y_hat_pred = predict_y_from_z(Z)
    pairs = [['subj_s1', 'subj_s2'], ['subj_s1', 'verb_s2'], ['subj_s1', 'obj_s2'], ['subj_s1', 'loc_s2'], ['subj_s1', 'clo_s2'],
             ['verb_s1', 'subj_s2'], ['verb_s1', 'verb_s2'], ['verb_s1', 'obj_s2'], ['verb_s1', 'loc_s2'], ['verb_s1', 'clo_s2'],
             ['obj_s1', 'subj_s2'], ['obj_s1', 'verb_s2'], ['obj_s1', 'obj_s2'], ['obj_s1', 'loc_s2'], ['obj_s1', 'clo_s2'],
             ['loc_s1', 'subj_s2'], ['loc_s1', 'verb_s2'], ['loc_s1', 'obj_s2'], ['loc_s1', 'loc_s2'], ['loc_s1', 'clo_s2'],
             ['clo_s1', 'subj_s2'], ['clo_s1', 'verb_s2'], ['clo_s1', 'obj_s2'], ['clo_s1', 'loc_s2'], ['clo_s1', 'clo_s2']]
    pairs_map = {'subj_s1': 'the subject of sentence 1',
                 'verb_s1': 'the verb of sentence 1',
                 'obj_s1': 'the object of sentence 1',
                 'loc_s1': 'the location of sentence 1',
                 'clo_s1': 'the clothing described in sentence 1',
                 'subj_s2': 'the subject of sentence 2',
                 'verb_s2': 'the verb of sentence 2',
                 'obj_s2': 'the object of sentence 2',
                 'loc_s2': 'the location of sentence 2',
                 'clo_s2': 'the clothing described in sentence 2',}
    reasons = list()
    if y_hat_pred == 'neutral':
        return 'The sentences are neutral'
    else:
        for i,z in enumerate(Z):
            if z == 'contradiction':
                if i in (0, 6, 12, 18, 24):
                    reasons += [f'{subphrases["string_" + pairs[i][0]].lower()} is not the same as {subphrases["string_" + pairs[i][1]].lower()}', ]
                else:
                    reasons += [f'if {pairs_map[pairs[i][0]]} is {subphrases["string_" + pairs[i][0]].lower()}, {pairs_map[pairs[i][1]]} cannot be {subphrases["string_" + pairs[i][1]].lower()}', ]
        if len(reasons) == 0:
            for i,z in enumerate(Z):
                if z == 'entailment':
                    if i in (0, 6, 12, 18, 24):
                        reasons += [f'{subphrases["string_" + pairs[i][0]].lower()} is the same as {subphrases["string_" + pairs[i][1]].lower()}', ]
                    else:
                        reasons += [f'if {pairs_map[pairs[i][0]]} is {subphrases["string_" + pairs[i][0]].lower()}, then {pairs_map[pairs[i][1]]} has to be {subphrases["string_" + pairs[i][1]].lower()}', ]
        return " and ".join(reasons)

In [97]:
for i in range(40):
    if y_hat[i] != y_hat_pred[i]:
        suffix = 'WRONG: '
    else:
        suffix = 'TRUE: '
    print(suffix + generate_explanation(z[i,:], dev_subphrases.iloc[i]))

TRUE: The sentences are neutral
WRONG: The sentences are neutral
WRONG: two women is the same as the men and if the subject of sentence 1 is two women, then the verb of sentence 2 has to be fighting and if the verb of sentence 1 is embracing, then the subject of sentence 2 has to be the men and embracing is the same as fighting
TRUE: a woman is the same as a woman and if the subject of sentence 1 is a woman, then the verb of sentence 2 has to be doing and if the subject of sentence 1 is a woman, then the object of sentence 2 has to be a cartwheel and if the verb of sentence 1 is doing, then the subject of sentence 2 has to be a woman and doing is the same as doing and if the verb of sentence 1 is doing, then the object of sentence 2 has to be a cartwheel and if the object of sentence 1 is a cartwheel, then the subject of sentence 2 has to be a woman and if the object of sentence 1 is a cartwheel, then the verb of sentence 2 has to be doing and a cartwheel is the same as a cartwheel and

In [77]:
human_z = pd.read_csv("manual_z_values.csv", sep=";").to_numpy()
for i in range(human_z.shape[0]):
    for j in range(human_z.shape[1]):
        if human_z[i,j] == "n":
            human_z[i,j] = "neutral"
        elif human_z[i,j] == "e":
            human_z[i,j] = "entailment"
        elif human_z[i,j] == "c":
            human_z[i,j] = "contradiction"

y_hat_human_pred = predict_y_from_z(human_z)

In [91]:
y_hat_human_pred[np.where(y_hat[:30] != y_hat_human_pred)]

array(['neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral'],
      dtype=object)

In [94]:
human_z[18,:]

array(['entailment', 'neutral', nan, 'neutral', nan, 'neutral',
       'entailment', nan, 'neutral', nan, nan, nan, nan, nan, nan,
       'neutral', 'neutral', nan, 'entailment', nan, nan, nan, nan, nan,
       nan], dtype=object)

In [88]:
dev_subphrases.iloc[np.where(y_hat[:30] != y_hat_human_pred)]

,Unnamed: 0,string_subj_s1,string_verb_s1,string_obj_s1,string_loc_s1,string_clo_s1,string_subj_s2,string_verb_s2,string_obj_s2,string_loc_s2,string_clo_s2
pairID,,,,,,,,,,,
4705552913.jpg#2r1e,1,Two women,embracing,NaN,NaN,NaN,Two woman,holding,packages,NaN,NaN
3948003394.jpg#1r1e,18,A woman,doing,a cartwheel,in the sand,bikini,A woman,doing,a cartwheel,NaN,NaN
3948003394.jpg#1r1c,19,A woman,doing,a cartwheel,in the sand,bikini,A woman,fixing,her home,NaN,NaN
5436250638.jpg#4r1e,43,Two doctors,perform,surgery on patient,on patient,NaN,Doctors,performing,surgery,NaN,NaN
485054073.jpg#0r1e,46,A white dog with long hair,catch,a red toy,NaN,NaN,An animal,catch,an object,NaN,NaN
7391785714.jpg#4r1e,50,Kids,are,NaN,on a amusement ride,NaN,Kids,ride,an amusement ride,NaN,NaN
6291644661.jpg#2r1e,62,A man in,stumbles,NaN,in a green jersey and rollerskates,NaN,The man,stumbles,NaN,in rollerskates,NaN
2447052614.jpg#0r1e,75,A man in a white t - shirt,takes,a picture,in a white t - shirt,white t shirt,A man,wearing,a white shirt,NaN,white shirt
5777129645.jpg#2r1n,86,The two farmers,working,NaN,on a piece of John Deere equipment,NaN,Men,working,NaN,on John Deere equipment,NaN


In [84]:
np.mean(y_hat_pred[:30] == y_hat_human_pred)

0.5333333333333333

In [85]:
np.mean(y_hat[:30] == y_hat_human_pred)

0.6

In [86]:
np.mean(y_hat_pred == y_hat)

0.4688632619439868

In [98]:
z

array([['entailment', 'entailment', 'entailment', ..., 'nan', 'nan',
        'nan'],
       ['entailment', 'entailment', 'entailment', ..., 'nan', 'nan',
        'nan'],
       ['entailment', 'entailment', 'nan', ..., 'nan', 'nan', 'nan'],
       ...,
       ['entailment', 'entailment', 'nan', ..., 'nan', 'nan', 'nan'],
       ['entailment', 'entailment', 'entailment', ..., 'nan', 'nan',
        'nan'],
       ['entailment', 'entailment', 'entailment', ..., 'nan', 'nan',
        'nan']], dtype='<U100')

In [105]:
z[39,:]

array(['entailment', 'entailment', 'neutral', 'nan', 'nan', 'neutral',
       'neutral', 'neutral', 'nan', 'nan', 'neutral', 'neutral',
       'neutral', 'nan', 'nan', 'neutral', 'neutral', 'neutral', 'nan',
       'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], dtype='<U100')

In [104]:
dev_subphrases.iloc[39]

Unnamed: 0                        43
string_subj_s1           Two doctors
string_verb_s1               perform
string_obj_s1     surgery on patient
string_loc_s1             on patient
string_clo_s1                    NaN
string_subj_s2               Doctors
string_verb_s2            performing
string_obj_s2                surgery
string_loc_s2                    NaN
string_clo_s2                    NaN
Name: 5436250638.jpg#4r1e, dtype: object